In [ ]:
import pandas as pd
import regex
from matplotlib import pyplot as plt
from datetime import datetime

from IPython.display import display # добавил явный импорт на всякий случай

In [ ]:
df = r'C:\GitRepo\My-training-Slubik-Stanislav\transactions_dirty.csv'
df = pd.read_csv(df)
display(df.head(10))

,account_number,account_type,date,operation,amount,balance_after,status
0,ACC-100001,checking,2025-09-27 22:17:26,deposit,921.0,2121.0,success
1,ACC-100001,checking,2025-09-27 22:17:26,deposit,607.0,2728.0,success
2,ACC-100001,checking,2025-09-28 22:17:26,deposit,488.0,3216.0,success
3,ACC-100001,checking,28/09/2025 22:17,deposit,129.0,3345.0,success
4,ACC-100001,checking,2025-09-29 22:17:26,deposit,880.0,4225.0,success
5,ACC-100001,checking,2025-09-29 22:17:26,withdraw,NaN,4039.0,success
6,ACC-100001,checking,2025-10-01 22:17:26,withdraw,352.0,3687.0,success
7,ACC-100001,checking,2025-10-01 22:17:26,withdraw,65.0,3622.0,success
8,ACC-100001,checking,2025-10-01 22:17:26,NaN,654.0,4276.0,success
9,ACC-100001,checking,2025-10-01 22:17:26,withdraw,245.0,4031.0,success


In [ ]:
from matplotlib.pyplot import isinteractive


class Account:
    _accont_counter = 1000 # счетчик для генерации уникальных номеров счёта
    
    def __init__(self, account_holder: str, balance: float = 0):
        
        self._account_holder = self._validate_holder_name(account_holder)
        self._balance = self._validate_balance(balance)
        
        self._account_number = f'ACC-{Account._accont_counter}'
        Account._accont_counter += 1
        
        self.operations_history = []
    
    
    
    @staticmethod # Проверка имени на соответствие паттерну как Имя Фамилия Отчество
    def _validate_holder_name(account_holder: str):
        account_holder = account_holder.title() # для лучшего соответствия паттерну привел к верхнему регистру
        pattern = r'^[A-ZА-Я][a-zа-я]+(-[A-ZА-Я][a-zа-я]+)?\s+[A-ZА-Я][a-zа-я]+\s+[A-ZА-Я][a-zа-я]$' # от а до я + латинские буквы, пробел(несколько) между ними "Имя Фамилия Отчество" допускается двойное имя
        if not isinstance(account_holder, str) or not regex.match(pattern, account_holder):
            raise ValueError(f'Неверный ввод: "{account_holder}". Пожалуйста, введите имя, фамилию и отчество.')
        return account_holder


    @staticmethod # Проверка баланса на соответсвие числовому значению и положительности
    def _validate_balance(balance: float):
        if not isinstance(balance, (int, float)) or balance < 0:
            raise ValueError('Баланс должен быть положительным')
        return float(balance)
        
        
        
        
        
        
        
        
        
        
        
        
        

